In [1]:
# import prereq.
from __future__ import absolute_import, division, print_function, unicode_literals
 
import numpy as np
import pandas as pd 
import tensorflow as tf

from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint # used for callback 


In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
model = tf.keras.models.load_model('97_935_93.h5')

In [4]:
def step_decay(epoch):
    initial_lrate = 0.00001
    return 1 / np.sqrt(epoch / 4 + 1) * initial_lrate
     



In [5]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(beta_1=0.99),
              metrics=['accuracy'])

lrate = LearningRateScheduler(step_decay)
mc = ModelCheckpoint('best_model_xd.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
callbacks = [es, mc,lrate]

In [6]:
X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')
XX = X.iloc[:int(X.shape[0]/4):,]
YY = Y.iloc[:int(X.shape[0]/4):,]

X_c = pd.read_csv('X_c.csv')
Y_c = pd.read_csv('Y_c.csv')

XX_c = X_c.iloc[0:int(X.shape[0]*1/4),:]
YY_c = Y_c.iloc[0:int(X.shape[0]*1/4),:]

X_t = pd.read_csv('X_t.csv')
Y_t = pd.read_csv('Y_t.csv')

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

data_type =  np.dtype(np.float64) # define datatype

x = np.asarray(XX.iloc[:,1:].to_numpy(), dtype=data_type)
y = np.asarray(YY.iloc[:,1].to_numpy(), dtype=data_type)
x_t = np.asarray(X_t.iloc[:,1:].to_numpy(), dtype=data_type)
y_t = np.asarray(Y_t.iloc[:,1].to_numpy(), dtype=data_type)
x_c = np.asarray(XX_c.iloc[:,1:].to_numpy(), dtype=data_type)
y_c = np.asarray(YY_c.iloc[:,1].to_numpy(), dtype=data_type)


# clean some memory
X = X_c = X_t = Y = Y_c = Y_t = None

x_train = x.reshape(x.shape[0], img_rows, img_cols, 1)
x_cross = x_c.reshape(x_c.shape[0], img_rows, img_cols, 1)
x_test = x_t.reshape(x_t.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols,1)

x_train = x_train.astype('float32')
x_cross = x_cross.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y, num_classes)
y_cross = keras.utils.to_categorical(y_c, num_classes)
y_test = keras.utils.to_categorical(y_t, num_classes)

In [7]:
model.evaluate(x_test, y_test)

350/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2505875767128808, 0.93142855]

In [8]:
model.fit(x_train,y_train, 
          epochs=50,
          batch_size=100,
          shuffle=True, 
          verbose=1,
         validation_data=(x_cross, y_cross), callbacks=callbacks)

Train on 69300 samples, validate on 1400 samples
Epoch 1/50
69200/69300 [============================>.] - ETA: 0s - loss: 0.0698 - accuracy: 0.9782
Epoch 00001: val_accuracy improved from -inf to 0.93714, saving model to best_model_xd.h5
69300/69300 [==============================] - 34s 488us/sample - loss: 0.0697 - accuracy: 0.9782 - val_loss: 0.2406 - val_accuracy: 0.9371
Epoch 2/50
69200/69300 [============================>.] - ETA: 0s - loss: 0.0698 - accuracy: 0.9787
Epoch 00002: val_accuracy improved from 0.93714 to 0.93857, saving model to best_model_xd.h5
69300/69300 [==============================] - 33s 473us/sample - loss: 0.0698 - accuracy: 0.9787 - val_loss: 0.2370 - val_accuracy: 0.9386
Epoch 3/50
69100/69300 [============================>.] - ETA: 0s - loss: 0.0681 - accuracy: 0.9789
Epoch 00003: val_accuracy did not improve from 0.93857
69300/69300 [==============================] - 32s 456us/sample - loss: 0.0680 - accuracy: 0.9789 - val_loss: 0.2424 - val_accuracy: 

In [9]:
model.evaluate(x_test, y_test)

350/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2560029524564743, 0.9342857]

In [10]:
#model.save('97_935_93_retrainde30.h5')

In [12]:
saved_model =  tf.keras.models.load_model('best_model_xd.h5')

In [13]:
saved_model.evaluate(x_train, y_train)

69300/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.010588889737622906, 0.9992208]

In [14]:
saved_model.evaluate(x_cross, y_cross)

1400/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.23698381185531617, 0.93857145]

In [15]:
saved_model.evaluate(x_test, y_test)

350/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2472333187716348, 0.9342857]